
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  16.01.2024

--- 

# Calculate Cold Start Emissions using HBEFA Emission factors

<!--Notebook description and usage information-->
In this notebook, HBEFA Cold-Start emission factors are applied to input from the visum model. <br>
HBEFA allows to separate cold-start emissions by: 
- Ambient temperature 
- Trip length 
- Parking hours (to determine how hot the engine is before the starting process)

HBEFA provides cold disaggregated cold start emissions where each of the parameters above can be set based on individual data. Since no Munich-specific information on trip lenght or parking hours is available, aggregated values for these categories are used. Temperature information is provided by a weather station in Munich and will be used for the whole region.<br>
It should be mentioned, that HBEFA just provides cold-start emissions for Personal Cars and Small Vans. They conistitue the main part of the overall number of cold starts in a city.


In [97]:
import sys

import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../utils')
import data_paths
import traffic_counts
from hbefa_cold_emissions import HbefaColdEmissions

from lmu_meteo_api import interface
from datetime import datetime
from gridding import GriddingEngine

## Import data

In [102]:
# import visum O-D matricies
visum_zones = gpd.read_file(data_paths.VISUM_FOLDER_PATH + "visum_zones.GPKG",
                            driver = 'GPKG')

# caclulate starts per squaremeter before gridding
visum_zones['PC_starts_per_squaremeter'] = visum_zones['PC_starts'] / visum_zones['geometry'].area
visum_zones['LCV_starts_per_squaremeter'] = visum_zones['LCV_starts'] / visum_zones['geometry'].area


munich_boarders = gpd.read_file(data_paths.MUNICH_BOARDERS_FILE).to_crs(25832)
visum_zones_clipped = gpd.clip(visum_zones, munich_boarders)
visum_zones_clipped = visum_zones_clipped.explode(ignore_index=True) # convert multipolygons to polygons

## Notebook functions

In [103]:
def annual_temperature_profile(start_date:datetime, 
                               end_date:datetime,
                               aggregate = 'H') -> pd.Series:
    """_summary_

    Args:
        year (int): _description_
        aggregate (str, optional): _description_. Defaults to 'H'.

    Returns:
        pd.Series: _description_
    """
    
    start_time = start_date.strftime('%Y-%m-%d') + 'T00-00-00'
    end_time = end_date.strftime('%Y-%m-%d') + 'T23-59-59'
    
    if datetime.strptime(end_time, '%Y-%m-%dT%H-%M-%S').date() > datetime.now().date():
        end_time = datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
    
    lmu_api = interface.meteo_data()
    data = lmu_api.get_meteo_data(parameters = ["air_temperature_2m"], 
                                station_id = 'MIM01', 
                                start_time = start_time, 
                                end_time = end_time)
    
    return (data.air_temperature_2m - 273.15).resample(aggregate).mean()

In [104]:
def get_hour_factor(cycles_obj,
                    date:str, 
                    hour:int, 
                    vehicle_class:str) -> float:
    """_summary_

    Args:
        cycles_obj (_type_): _description_
        date (str): _description_
        hour (int): _description_
        vehicle_class (str): _description_

    Returns:
        float: _description_
    """
    return cycles_obj.get_hourly_scaling_factors(date).loc[vehicle_class].iloc[hour]

## Initialize objects and prepare parameters table

In [5]:
# import trafic datasets
cycles = traffic_counts.TrafficCounts()

# activity on Local/Collector roads was selected to be most 
# representative to scale the number of cold starts
# reduce to 2019 activity
activity = cycles.annual_cycles['Local/Collector'].loc['2019-01-01' : '2019-12-31']

# instanciate cold start emission object
cs_obj = HbefaColdEmissions()

# download temperature data for 2019
temperature = annual_temperature_profile(start_date=datetime(2019,1,1),
                                         end_date=datetime(2019,12,31)) 

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_ColdStart_Vehcat_ColdStart.XLS


In [6]:
parameters = pd.DataFrame(index = pd.date_range(start='2019-01-01 00:00:00',
                                                end = '2019-12-31 23:00:00',
                                                freq='1h'))
parameters['date'] = parameters.index.date
parameters['hour'] = parameters.index.hour
parameters['temperature'] = temperature
parameters['daily_value'] = activity
parameters['daily_value'] = parameters['daily_value'].ffill(limit=24)

parameters['cycle_PC'] = parameters.apply(lambda row: get_hour_factor(cycles,
                                                                           row['date'].strftime('%Y-%m-%d'),
                                                                           row['hour'],
                                                                           'PC'), axis = 1)
parameters['cycle_LCV'] = parameters.apply(lambda row: get_hour_factor(cycles,
                                                                           row['date'].strftime('%Y-%m-%d'),
                                                                           row['hour'],
                                                                           'LCV'), axis = 1)

parameters['hour_factor_PC'] = parameters['cycle_PC'] * parameters['daily_value']
parameters['hour_factor_LCV'] = parameters['cycle_LCV'] * parameters['daily_value']

In [120]:
#calculate daily coldstarts in Munich
daily_PC_starts = (visum_zones_clipped['PC_starts_per_squaremeter'] * visum_zones_clipped.area).sum()
daily_LCV_starts = (visum_zones_clipped['LCV_starts_per_squaremeter'] * visum_zones_clipped.area).sum()

em_list_pc = list()
em_list_lcv = list()

for idx, row in parameters.iterrows():
    
    # get emission factors
    em_PC = cs_obj.calculate_emission_daily(hourly_temperature=row['temperature'],
                                            vehicle_class='PC',
                                            year = idx.year)
    
    em_LCV = cs_obj.calculate_emission_daily(hourly_temperature=row['temperature'],
                                             vehicle_class='LCV',
                                             year = idx.year)
    
    # hourly number of starting procedures
    hourly_PC_starts = daily_PC_starts * row['hour_factor_PC']
    hourly_LCV_starts = daily_LCV_starts * row['hour_factor_LCV']
        
    # hourly emissions
    em_list_pc.append((em_PC * hourly_PC_starts))
    em_list_lcv.append((em_LCV * hourly_LCV_starts))

In [122]:
em_sum_PC = pd.concat(em_list_pc, axis = 1).sum(axis=1) * 1e-6
em_sum_LCV = pd.concat(em_list_pc, axis = 1).sum(axis=1) * 1e-6

In [124]:
em_sum_fin = pd.concat({'PC':em_sum_PC, 'LCV': em_sum_LCV}, axis = 1)
em_sum_fin['SUM'] = em_sum_fin['PC'] + em_sum_fin['LCV']

,PC,LCV
Component,,
NOx,7.407259e+01,7.407259e+01
FC,8.286776e+03,8.286776e+03
FC_MJ,3.521738e+02,3.521738e+02
PM,1.623096e+00,1.623096e+00
PN,7.292881e+14,7.292881e+14
CO2(rep),2.454842e+04,2.454842e+04
CO2(total),2.571164e+04,2.571164e+04
NO2,-4.688031e-01,-4.688031e-01
HC,3.455170e+02,3.455170e+02


In [118]:
em_list_lcv.append(em_list_pc)

In [52]:
parameters.groupby('date').agg({'daily_value':lambda r: r.iloc[0]})

,daily_value
date,
2019-01-01,NaN
2019-01-02,0.009283
2019-01-03,0.749901
2019-01-04,0.739115
2019-01-05,0.542371
...,...
2019-12-27,0.692002
2019-12-28,0.602442
2019-12-29,0.455192


In [ ]:
# import TNO 100m grid
path = data_paths.TNO_100M_GRID
in_grid = gpd.read_file(path)
in_grid = in_grid.to_crs(25832)

grd = GriddingEngine(input_grid=in_grid)